In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import uniform, norm
from bayesian_estimation import BayesianEstimation, LogLikelihoods

# OC Dam

## Load systematic data

In [ ]:
annual_maxima_csv = r"C:\ISYE6420\Homework\Project\data\OC_Dam.csv"

df = pd.read_csv(annual_maxima_csv)
df["zstd"] = -norm.ppf(df["Plotting_Position"])

# convert flow to array
data = df["Flow"].to_xarray()

time_index = np.arange(len(data))

## Stationary

In [ ]:
# Compute the prior limits
mu_min, mu_max = 0, 3
sigma_min, sigma_max = 0, 2
gamma_min, gamma_max = -2, 2

# Define the prior distributions
priors = [
    lambda: uniform.rvs(loc=mu_min, scale=mu_max - mu_min),
    lambda: uniform.rvs(loc=sigma_min, scale=sigma_max - sigma_min),
    lambda: uniform.rvs(loc=gamma_min, scale=gamma_max - gamma_min)
]

prior_limits = [
    (mu_min, mu_max),
    (sigma_min, sigma_max),
    (gamma_min, gamma_max)
]

log_likelihood_func = LogLikelihoods(data).lp3

bayesian_estimation_lp3 = BayesianEstimation(
    data=data, 
    log_likelihood_func=log_likelihood_func, 
    prior=priors, 
    prior_limits=prior_limits, 
    seed = 253
    )

# Run the DEMCz sampler
samples, acceptance_rates = bayesian_estimation_lp3.demcz_sampler(
    num_chains=5, 
    iterations=44000, 
    burn_in=4000, 
    jump=0.97163, 
    jump_threshold=0.1,
    noise=1e-3,
    snooker_threshold=0.1,
    thinning_interval=20
    )
np.save(r'C:\ISYE6420\Homework\Project\data\OCD\OCD_bayesian_stationary_lp3_samples.npy', samples)

# Display the acceptance rates
print("Acceptance rates:\n")
print(acceptance_rates)

# Display the summary statistics
summaries_lp3 = bayesian_estimation_lp3.calculate_log_likelihoods(
    variable=["mu", "sigma", "gamma"]
    )

summaries_lp3.to_csv(r'C:\ISYE6420\Homework\Project\data\OCD\OCD_bayesian_stationary_lp3_summaries.csv', index=False)

posterior_mode = summaries_lp3.loc[summaries_lp3['LogLikelihood'].idxmax()]
    
print("\nPosterior mode:\n")
print(posterior_mode)

## Non-stationary

### $\mu$: linear trend

In [ ]:
# Compute the prior limits
beta_0_min, beta_0_max = 0, 3  # Prior for beta_0
beta_1_min, beta_1_max = -1, 1  # Prior for beta_1 (slope of mu_t)
sigma_min, sigma_max = 0, 2
gamma_min, gamma_max = -2, 2

# Define the prior distributions
priors = [
    lambda: uniform.rvs(loc=beta_0_min, scale=beta_0_max - beta_0_min),
    lambda: uniform.rvs(loc=beta_1_min, scale=beta_1_max - beta_1_min),
    lambda: uniform.rvs(loc=sigma_min, scale=sigma_max - sigma_min),
    lambda: uniform.rvs(loc=gamma_min, scale=gamma_max - gamma_min)
]

prior_limits = [
    (beta_0_min, beta_0_max),
    (beta_1_min, beta_1_max),
    (sigma_min, sigma_max),
    (gamma_min, gamma_max)
]

log_likelihood_func_linear = lambda theta: LogLikelihoods(data).lp3_mu(
    theta, 
    time_index, 
    model_type = "linear"
    )

bayesian_estimation_lp3 = BayesianEstimation(
    data=data, 
    log_likelihood_func=log_likelihood_func_linear, 
    prior=priors, 
    prior_limits=prior_limits, 
    seed = 253
    )

# Run the DEMCz sampler
samples, acceptance_rates = bayesian_estimation_lp3.demcz_sampler(
    num_chains=5, 
    iterations=44000, 
    burn_in=4000, 
    jump=0.84145, 
    jump_threshold=0.1,
    noise=1e-3,
    snooker_threshold=0.1,
    thinning_interval=20
    )
np.save(r'C:\ISYE6420\Homework\Project\data\OCD\OCD_bayesian_linear_mu_lp3_samples.npy', samples)

# Display the acceptance rates
print("Acceptance rates:\n")
print(acceptance_rates)

summaries_lp3 = bayesian_estimation_lp3.calculate_log_likelihoods(
    variable=["beta_0", "beta_1", "sigma", "gamma"]
    )

summaries_lp3.to_csv(r'C:\ISYE6420\Homework\Project\data\OCD\OCD_bayesian_linear_mu_lp3_summaries.csv', index=False)

posterior_mode = summaries_lp3.loc[summaries_lp3['LogLikelihood'].idxmax()]
    
print("\nPosterior mode:\n")
print(posterior_mode)

Acceptance rates:

[0.44509091 0.44315909 0.44470455 0.44506818 0.44620455]

Posterior mode:

beta_0             1.868633
beta_1            -0.010786
sigma              0.613886
gamma             -0.047613
LogLikelihood   -504.492833
Name: 4535, dtype: float64


### $\mu$: exponential trend

In [ ]:
# Compute the prior limits
beta_0_min, beta_0_max = 0, 3  # Prior for beta_0
beta_1_min, beta_1_max = -1, 1  # Prior for beta_1 (slope of mu_t)
sigma_min, sigma_max = 0, 2
gamma_min, gamma_max = -2, 2

# Define the prior distributions
priors = [
    lambda: uniform.rvs(loc=beta_0_min, scale=beta_0_max - beta_0_min),
    lambda: uniform.rvs(loc=beta_1_min, scale=beta_1_max - beta_1_min),
    lambda: uniform.rvs(loc=sigma_min, scale=sigma_max - sigma_min),
    lambda: uniform.rvs(loc=gamma_min, scale=gamma_max - gamma_min)
]

prior_limits = [
    (beta_0_min, beta_0_max),
    (beta_1_min, beta_1_max),
    (sigma_min, sigma_max),
    (gamma_min, gamma_max)
]

log_likelihood_func_linear = lambda theta: LogLikelihoods(data).lp3_mu(
    theta, 
    time_index, 
    model_type = "exponential"
    )

bayesian_estimation_lp3 = BayesianEstimation(
    data=data, 
    log_likelihood_func=log_likelihood_func_linear, 
    prior=priors, 
    prior_limits=prior_limits, 
    seed = 253
    )

# Run the DEMCz sampler
samples, acceptance_rates = bayesian_estimation_lp3.demcz_sampler(
    num_chains=5, 
    iterations=44000, 
    burn_in=4000, 
    jump=0.84145, 
    jump_threshold=0.1,
    noise=1e-3,
    snooker_threshold=0.1,
    thinning_interval=20
    )
np.save(r'C:\ISYE6420\Homework\Project\data\OCD\OCD_bayesian_exponential_mu_lp3_samples.npy', samples)

# Display the acceptance rates
print("Acceptance rates:\n")
print(acceptance_rates)

summaries_lp3 = bayesian_estimation_lp3.calculate_log_likelihoods(
    variable=["beta_0", "beta_1", "sigma", "gamma"]
    )

summaries_lp3.to_csv(r'C:\ISYE6420\Homework\Project\data\OCD\OCD_bayesian_exponential_mu_lp3_summaries.csv', index=False)

posterior_mode = summaries_lp3.loc[summaries_lp3['LogLikelihood'].idxmax()]
    
print("\nPosterior mode:\n")
print(posterior_mode)

Acceptance rates:

[0.44784091 0.44343182 0.43793182 0.44493182 0.44227273]

Posterior mode:

beta_0             1.934486
beta_1            -0.008113
sigma              0.618743
gamma             -0.034503
LogLikelihood   -504.935919
Name: 1639, dtype: float64
